In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# 假設有一個事故資料表，包含經緯度數據
data = {
    '日期': ['2023-01-01', '2023-01-02', '2023-01-03'],
    '天氣': ['晴天', '雨天', '陰天'],
    '地區': ['北部', '中部', '南部'],
    '經度': [121.5, 120.5, 121.0],
    '緯度': [25.0, 24.0, 23.5],
    '事故數': [10, 5, 8]
}
df = pd.DataFrame(data)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id='weather-filter',
        options=[{'label': weather, 'value': weather} for weather in df['天氣'].unique()],
        value=df['天氣'].unique().tolist(),
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Checklist(
        id='region-filter',
        options=[{'label': region, 'value': region} for region in df['地區'].unique()],
        value=df['地區'].unique().tolist(),
        labelStyle={'display': 'inline-block'}
    ),
    dcc.DatePickerRange(
        id='date-picker',
        start_date=df['日期'].min(),
        end_date=df['日期'].max(),
        display_format='YYYY-MM-DD'
    ),
    dcc.Graph(id='map'),
    html.Div(id='filtered-data')
])

@app.callback(
    [Output('map', 'figure'),
     Output('filtered-data', 'children')],
    [Input('weather-filter', 'value'),
     Input('region-filter', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_output(selected_weathers, selected_regions, start_date, end_date):
    filtered_df = df[
        (df['天氣'].isin(selected_weathers)) &
        (df['地區'].isin(selected_regions)) &
        (df['日期'] >= start_date) &
        (df['日期'] <= end_date)
    ]
    
    fig = px.scatter_mapbox(filtered_df, lat='緯度', lon='經度', color='事故數',
                            hover_name='地區', hover_data=['日期', '天氣'],
                            zoom=5, height=500)
    fig.update_layout(mapbox_style="open-street-map")
    
    return fig, filtered_df.to_dict('records')

if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Sample data
data = {
    '日期': ['2023-01-01', '2023-01-02', '2023-01-03'],
    '天氣': ['晴天', '雨天', '陰天'],
    '地區': ['北部', '中部', '南部'],
    '經度': [121.5, 120.5, 121.0],
    '緯度': [25.0, 24.0, 23.5],
    '事故數': [10, 5, 8]
}
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Checklist(
                id='weather-filter',
                options=[{'label': weather, 'value': weather} for weather in df['天氣'].unique()],
                value=df['天氣'].unique().tolist(),
                labelStyle={'display': 'inline-block'}
            ),
            dcc.Checklist(
                id='region-filter',
                options=[{'label': region, 'value': region} for region in df['地區'].unique()],
                value=df['地區'].unique().tolist(),
                labelStyle={'display': 'inline-block'}
            ),
            dcc.DatePickerRange(
                id='date-picker',
                start_date=df['日期'].min(),
                end_date=df['日期'].max(),
                display_format='YYYY-MM-DD',
                start_date_placeholder_text="選擇開始日期",
                end_date_placeholder_text="選擇結束日期",
            ),
            dcc.Graph(id='map', style={'height': '500px'}),
        ], style={'width': '100%', 'display': 'inline-block', 'vertical-align': 'top'}),
    ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'}),
    
    html.Div([
        html.H3('Filtered Data'),
        html.Ul(id='filtered-data-list'),
    ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'}),
    
    html.Div([
        dcc.Graph(id='scatter-plot', style={'height': '500px'}),
    ], style={'width': '100%', 'display': 'inline-block', 'vertical-align': 'top'}),
])

@app.callback(
    [Output('map', 'figure'),
     Output('filtered-data-list', 'children'),
     Output('scatter-plot', 'figure')],
    [Input('weather-filter', 'value'),
     Input('region-filter', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_output(selected_weathers, selected_regions, start_date, end_date):
    # Filter data based on user inputs
    filtered_df = df[
        (df['天氣'].isin(selected_weathers)) &
        (df['地區'].isin(selected_regions)) &
        (df['日期'] >= start_date) &
        (df['日期'] <= end_date)
    ]
    
    print(filtered_df)  # Print filtered data for debugging
    
    # Convert filtered data to a list of html.Li elements
    filtered_data_list = [
        html.Li(f"日期: {row['日期']}, 天氣: {row['天氣']}, 事故數: {row['事故數']}")
        for idx, row in filtered_df.iterrows()
    ]
    
    # Update map figure
    if filtered_df.empty:
        fig_map = px.scatter_mapbox(lat=[0], lon=[0], zoom=1)
        fig_map.update_layout(mapbox_style="open-street-map")
    else:
        fig_map = px.scatter_mapbox(filtered_df, lat='緯度', lon='經度', color='事故數',
                                    hover_name='地區', hover_data=['日期', '天氣'],
                                    zoom=5, height=500)
        fig_map.update_layout(mapbox_style="open-street-map")
    
    # Update scatter plot figure (dummy data for illustration)
    fig_scatter = px.scatter(df, x='日期', y='事故數', color='天氣', hover_data=['地區'], title='Scatter Plot')
    
    print(fig_scatter)  # Print fig_scatter for debugging
    
    return fig_map, filtered_data_list, fig_scatter

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


           日期  天氣  地區     經度    緯度  事故數
0  2023-01-01  晴天  北部  121.5  25.0   10
1  2023-01-02  雨天  中部  120.5  24.0    5
2  2023-01-03  陰天  南部  121.0  23.5    8
Figure({
    'data': [{'customdata': array([['北部']], dtype=object),
              'hovertemplate': '天氣=晴天<br>日期=%{x}<br>事故數=%{y}<br>地區=%{customdata[0]}<extra></extra>',
              'legendgroup': '晴天',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '晴天',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2023-01-01'], dtype=object),
              'xaxis': 'x',
              'y': array([10]),
              'yaxis': 'y'},
             {'customdata': array([['中部']], dtype=object),
              'hovertemplate': '天氣=雨天<br>日期=%{x}<br>事故數=%{y}<br>地區=%{customdata[0]}<extra></extra>',
              'legendgroup': '雨天',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
  